## Hyperparameter search

Source: https://github.com/itdxer/neupy/blob/master/notebooks/Hyperparameter%20optimization%20for%20Neural%20Networks.ipynb
and http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html#id14

```python
Hyperparams to train:
1. lamda
2. number of hidden layers
3. number of hidden units
3. starter learning rate
5. use_sigmoid_h1
6. use_sigmoid_h2```

In [1]:
import time
import os
import sys
sys.path = sys.path[1:]
sys.path.insert(0, os.path.join(os.getcwd(), "helper_scripts"))  # pretending we are running main.py in jupyter

from global_variables_final_for_git import Global_Vars
from prep_for_model_for_git import Model_preparation
import pandas as pd

import hyperopt
from hyperopt import fmin
import HPO_helper
from HPO_helper import uniform_int, loguniform_int, tpe_method, get_parameter_space_forHPO
from tensorflow_model_for_git import Tensorflow_model

start_time = time.time()
if __name__=="__main__":
    class Args(object):
        def __init__(self):
            self.gene = "ARPP21"
            self.distance = 200
            self.use_tad_info = True
            self.pcc_lowerlimit_to_filter_dhss = 0.25
            self.take_log2_tpm = True
            self.filter_tfs_by = "zscore" # or "pcc"
            self.lowerlimit_to_filter_tfs = 4.75
            self.take_this_many_top_fts = 15  # all dhss/tfs will already be filtered by pcc(or zscore)
            self.init_wts_type = "corr"
            self.outputDir = "/Users/Dinesh/Dropbox/Github/predicting_gex_with_nn_git/Output/testing/ARPP21"
            self.use_random_DHSs = False
            self.use_random_TFs = False
            self.max_iter = 500

    args = Args()
    gv = Global_Vars(args, args.outputDir)  # note this takes in new_output_dir as well in .py scripts
    mp = Model_preparation(gv)

2018-01-14 20:28:09,308: global_variables_final_for_git: INFO    : Setting up the DNase-seq dataframe and gene expression vector..
2018-01-14 20:28:10,162: global_variables_final_for_git: ERROR   : The gene name is not found. Double check the name.


Exception: The gene name is not found. Double check the name?

In [ ]:
'''Run HPO on differen train/test splits'''
for test_idx in range(0, 19):
    # test_idx = 5  # 5 corresponds to ES_deriv group
    if (test_idx == 4): # corresponds to val_group of "ENCODE2012"
        continue
    start_time = time.time()
    tm = Tensorflow_model(gv, mp, test_eid_group_index=test_idx)
    trials = hyperopt.Trials()

    best_params = hyperopt.fmin(
        tm.train_tensorflow_nn,
        trials=trials,
        space=get_parameter_space_forHPO(tm.trainX),
        algo=tpe_method,     # Set up TPE for hyperparameter optimization
        max_evals=5,     # Maximum number of iterations. Basically it trains at most 200 networks before choose the best one.
    )

    med_pc_test_error = tm.plot_scatter_performance(trials, gv, index=None)
    tm.logger.info("Test Group {}:{}, Median Test Percentage Error: {}, Best Params: {}".format(
            tm.test_eid_group_index, tm.test_eid_group,
            round(med_pc_test_error, 4), best_params))

    if (test_idx == 6): 
         break
    del tm, trials, best_params
    
print("Total time taken: {}".format(time.time() - start_time))

In [ ]:
# - EOF - 